https://colab.research.google.com/drive/13ErkLg5FZHIbnUGZRkKlL-9WNCNQPIow#scrollTo=H5t4_7HdjYEE


1.   pip install transformers 






In [1]:
import pandas as pd
import matplotlib
import numpy as np
import torch
import transformers
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification

In [6]:
batch_size = 6
max_sequence_length = 512
learning_rate = 2e-5
number_of_epochs = 1
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels = 6)
# choosing Adam optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=1e-08)
# we do not have one-hot vectors, we can use sparce categorical cross entropy and accuracy
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
df = pd.read_csv('train.csv')
df

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5
3,003885A45F42,The best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5
...,...,...,...,...,...,...,...,...
3906,FFD29828A873,I believe using cellphones in class for educat...,2.5,3.0,3.0,3.5,2.5,2.5
3907,FFD9A83B0849,"Working alone, students do not have to argue w...",4.0,4.0,4.0,4.0,3.5,3.0
3908,FFDC4011AC9C,"""A problem is a chance for you to do your best...",2.5,3.0,3.0,3.0,3.5,3.0
3909,FFE16D704B16,Many people disagree with Albert Schweitzer's ...,4.0,4.5,4.5,4.0,4.5,4.5


In [7]:
def get_label(df):
  df_cohesion = df[["text_id", "full_text", "cohesion"]]
  df_cohesion['rating'] = np.zeros(len(df_cohesion))
  for i in range(1, len(df_cohesion)):
    score = df_cohesion.loc[i, 'cohesion']
    if score >= 3:
        df_cohesion.loc[i, 'rating'] = 1
    else:
        df_cohesion.loc[i, 'rating'] = 0

  return df_cohesion

#df_cohesion = get_label(df)
df_cohesion = df[['text_id', 'full_text', 'cohesion']]

https://towardsdatascience.com/how-to-apply-transformers-to-any-length-of-text-a5601410af7f

In [9]:
def map_example_to_dict(input_ids, attention_masks, token_type_ids, label):
  return {
      "input_ids": input_ids,
      "token_type_ids": token_type_ids,
      "attention_mask": attention_masks,
  }, label

def convert_example_to_feature(review): #make dataset to be a square 
  return tokenizer.encode_plus(review,
                add_special_tokens = True, # add [CLS], [SEP]
                max_length = 512, # max length of the text that can go to BERT
                pad_to_max_length = True, # add [PAD] tokens 按照最大长度补齐
                return_attention_mask = True, # add attention mask to not focus on pad tokens
              )
  
def encode_examples():  
  max_len = 0
  input_ids_list = []
  token_type_ids_list = []
  attention_mask_list = []
  label_list = []

  sentences = df_cohesion["full_text"].values
  #labels_raw = df_cohesion["rating"].values
  labels_raw = df_cohesion["cohesion"].values

  for k in range(len(sentences)):
      sent = sentences[k]
      tokens = convert_example_to_feature(sent)
      input_ids_list.append(tokens['input_ids'])
      token_type_ids_list.append(tokens['token_type_ids'])
      attention_mask_list.append(tokens['attention_mask'])
      label_list.append([labels_raw[k]])

  dataset = tf.data.Dataset.from_tensor_slices((input_ids_list, attention_mask_list, 
                                             token_type_ids_list, label_list)).map(map_example_to_dict)
  return dataset

ds_train_encoded = encode_examples().shuffle(1000).batch(batch_size)
ds_test_encoded = encode_examples().shuffle(100).batch(batch_size)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2310: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [10]:
bert_history = model.fit(ds_train_encoded, epochs=number_of_epochs, validation_data=ds_test_encoded)

652/652 [==============================] - 124s 157ms/step - loss: 0.9743 - accuracy: 0.2807 - val_loss: 0.7969 - val_accuracy: 0.3278


In [30]:
def getLongestParagraph(df_cohesion):
  lengths = df_cohesion["full_text"].str.len()
  argmax = np.where(lengths == lengths.max())[0]
  return df_cohesion.iloc[argmax]["full_text"].values[0]

def predict_grade(txt):
  tokens = tokenizer.encode(txt, truncation=True, padding=True, return_tensors='tf')
  tf_output= model.predict(tokens)[0]
  tf_prediction = tf.nn.softmax(tf_output, axis=1)
  numpy_prediction = tf_prediction.numpy()
  txt_label = tf.argmax(numpy_prediction, axis=1)
  return txt_label.numpy()[0]

#txt = getLongestParagraph(df_cohesion)
txt = df_cohesion["full_text"][4]
print(predict_grade(txt))

1/1 [==============================] - 0s 43ms/step
2


In [20]:
def getLongestParagraph(df_cohesion):
  lengths = df_cohesion["full_text"].str.len()
  argmax = np.where(lengths == lengths.max())[0]
  return df_cohesion.iloc[argmax]["full_text"].values[0]

def getGrade(numpy_prediction):
  return int(numpy_prediction[0][1]*10) / 2

def predict_grade(txt):
  tokens = tokenizer.encode(txt, truncation=True, padding=True, return_tensors='tf')
  tf_output= model.predict(tokens)[0]
  tf_prediction = tf.nn.softmax(tf_output, axis=1)
  numpy_prediction = tf_prediction.numpy()
  txt_grading = getGrade(numpy_prediction)
  return txt_grading

txt = getLongestParagraph(df_cohesion)
predict_grade(txt)

1/1 [==============================] - 0s 45ms/step


0.0

In [18]:
def computeLoss100(df_cohesion, model):
    loss = 0
    for index, row in df_cohesion.head(100).iterrows():
        txt = row["full_text"]
        predicted_cohesion_score = predict_grade(txt)
        loss = loss + (row["cohesion"] - predicted_cohesion_score) ** 2
    return loss
    
loss = computeLoss100(df_cohesion, model)

1/1 [==============================] - 0s 44ms/step


TypeError: ignored

In [ ]:
loss

213.0

In [ ]:
def breakintochunks(input_id_chunks, mask_chunks, label): #Not useful by now
    input_id_chunks_ = list(input_id_chunks)
    mask_chunks_ = list(mask_chunks)

    for i in range(len(input_id_chunks_)):
      input_id_chunks_[i] = torch.cat([
          torch.Tensor([101]), input_id_chunks_[i], torch.Tensor([102])
          ])
      mask_chunks_[i] = torch.cat([
          torch.Tensor([1]), mask_chunks_[i], torch.Tensor([1])
          ])
      
      pad_len = max_sequence_length - input_id_chunks_[i].shape[0]
      if pad_len > 0:
        input_id_chunks_[i] = torch.cat([
            input_id_chunks_[i], torch.Tensor([0] * pad_len)
        ])

        mask_chunks_[i] = torch.cat([
            mask_chunks_[i], torch.Tensor([0] * pad_len)
        ])
      #print(len(input_id_chunks_[i]), end = ' ')
    #print()

    input_id = torch.stack(input_id_chunks_)
    attention_mask = torch.stack(mask_chunks_)
    labels_nparray = np.full(len(input_id_chunks_), label)
    labels_tensor = torch.from_numpy(labels_nparray)
    #print(len(input_id), len(attention_mask), len(labels_tensor))
    return input_id, attention_mask, labels_tensor